# Przybliżanie stałych i funkcji za pomocą sum szeregów

Czym jest programowanie komputera?

* Można by przyjąć, że jest to bardzo podobne do uczenia. 
* Programista analizuje czynność lub inną składową, a następnie uczy komputer jak ją wykonać, lub w czym leży istota rzeczy. 
* Jeśli umie coś wykonać programista - może nauczyć tego komputer. 
* A tego co umie komputer musiał nauczyć go pewien programista.

Jeśli byśmy chcieli nauczyć komputer obliczać rozwiązanie równania liniowego $ax+b = y$ wystarczy napisać

In [1]:
def solve_linear(a,b,y):
    if a==0 and b != y :
        return None
    elif a==0 and b == y:
        return (float('-inf'), float('inf'))
    else:
        return (y-b) / a

solve_linear(1,3,2)

-1.0

Przyjrzyjmy się teraz trudniejszemu zadaniu. Jak obliczyć wartość $e^2$ ? Ha trywialne !! Wystarczy napisać


In [2]:
import math
math.exp(2.0)

7.38905609893065

Zauważmy jednak dwie istotne rzeczy w takim podejściu

* Liczba $e^2$ jest liczbą niewymierną - zatem na pewno nie jest dokładnie tą liczbą powyżej. Komputer przecież nigdy nie będzie mieć dość pamięci, aby zaprezentować wszystkie cyfry (jest ich oczywiście nieskończenie wiele) liczby niewymiernej. Zatem oczywiście jest to jedynie pewne przybliżenie prawdziwej wartości (w sumie niewiemy nawet jak bardzo dokładne)
* Ponadto jeśli w pewnej bibliotece istnieje funkcja exp, to ktoś ją musiał napisać. Ktoś musiał nauczyć komputer jak policzyć tę wartość (to przybliżenie). Najciekawsze jednak jest to, że w sumie w tej chwili nie wiemy jak je policzyć samodzielnie na kartce. (**Najwięksi spryciarze by pewnie zaczęli od przypomnienia, że $2.71 \leq e \leq 2.72$ policzyli kwadrat obustron, jako że $e^x$ jest rosnąca to otrzymaliby pewne przybliżenie. Jednak nawet by to kontynuować trzeba by mieć lepsza dokładność liczby $e$ **)

W tej lekcji pokażemy jak działają (jakie techniki zastosowali programisci przy obliczeniu/pisaniu) obliczenia skomplikowanych funkcji elementarnych oraz w jaki sposób zostały policzone przybliżenia ważnych stałych.



## Plan warsztatu

Poszukiwanie przybliżeń najważniejszych stałych

* Liczby e
* Liczby $\pi$
* Liczby $\sqrt{2}$
* Funkcji na przykładzie $\sin$

Kody możecie Państwo uruchamiać sobie samodzielnie po pobraniu repozytorium z 

[https://github.com/centrum-mistrzostwa-informatycznego/cmi-edition3]()

# Liczba $e$

## Granica ciągu

Naszą przygodę z przybliżaniem liczb rozpoczniemy od liczby $e$. Jak wiadomo z definicji matematycznej liczba ta spełnia warunek

$$
e = \lim_{n \to \infty} \left( 1 + \frac{1}{n} \right)^n.
$$

Możemy zatem pokusić się o obliczanie kolejnych wartości ciągu i obserwowanie obliczeń.

In [3]:
def e_ciagiem(n):
    return (1.0+1.0/n)**n # 1 + 1 + n-1 = n+1

for i in range(1,20):
    print(f'Przybliżenie {i} wynosi {e_ciagiem(i)}')

Przybliżenie 1 wynosi 2.0
Przybliżenie 2 wynosi 2.25
Przybliżenie 3 wynosi 2.37037037037037
Przybliżenie 4 wynosi 2.44140625
Przybliżenie 5 wynosi 2.4883199999999994
Przybliżenie 6 wynosi 2.5216263717421135
Przybliżenie 7 wynosi 2.546499697040712
Przybliżenie 8 wynosi 2.565784513950348
Przybliżenie 9 wynosi 2.5811747917131984
Przybliżenie 10 wynosi 2.5937424601000023
Przybliżenie 11 wynosi 2.6041990118975287
Przybliżenie 12 wynosi 2.613035290224676
Przybliżenie 13 wynosi 2.6206008878857308
Przybliżenie 14 wynosi 2.6271515563008685
Przybliżenie 15 wynosi 2.6328787177279187
Przybliżenie 16 wynosi 2.6379284973666
Przybliżenie 17 wynosi 2.64241437518311
Przybliżenie 18 wynosi 2.6464258210976865
Przybliżenie 19 wynosi 2.650034326640442


In [4]:
import math
math.e

2.718281828459045

In [5]:
e_ciagiem(10000)

2.7181459268249255

Widzimy zatem, że nawet biorąc 1000 pierwszy przybliżeń - nie udało nam się uzyskać przybliżenia lepszego niż rzędu $0.01$

In [6]:
print(f'Przybliżenie {1000} wynosi {e_ciagiem(1000)}')

Przybliżenie 1000 wynosi 2.7169239322355936


Zastanówmy się też chwilę nad złożonością obliczeniową naszego rozwiązania. Początkowo wykonujemy jedno dzielenie oraz jedno dodawania. Czyli 2 operacje. Potem jednak dokonujemy potęgowania. Rozsądnie jest przyjąć, że potęgowanie oznaczać będzie $n-1$ mnożeń. Co łącznie daje $n+1$ operacji zmiennoprzecinkowych.
```{python}
def e_ciagiem(n):
    return (1.0+1.0/n)**n # 1 + 1 + n-1 = n+1
```

## Wzór Taylora

W drugim podejściu wykorzystywać będziemy słynne twierdzenie o wzorze Taylora. Pozwala ono odnajdywać wielomiany bliskie funkcją posiadającym odpowiednio dobrą klasę różniczkowalności. Wykorzystamy tu zatem własność, że $e = exp(1)$. Najpierw przypomnijmy samo sformułowanie twierdzenia:

**Twierdzenie Taylora**
*Niech $[a,b] \subset \mathbb{R} $ pewien przedział oraz $ f: [a,b] \to \mathbb{R}$ będzie funkcją $(n+1)$ różniczkowalną na przedziale $[a,b]$ w sposób ciągły. Wówczas dla każdego punktu z przedziału $(a,b)$ spełniony jest następujący wzór
$$
f(x) = f(a) + \frac{x-a}{1!} f^{(1)}(a) + \frac{(x-a)^2}{2!} f^{(2)}(a) + \ldots + \frac{(x-a)^n}{n!} + \frac{(x-a)^{n+1}}{(n+1)!} f^{(n+1)} (\xi),
$$
gdzie $\xi \in [x,a]$ (gdy $x < a$ lub do $[a,x]$ jeśli $x >a$)*

W efekcie stosowania powyższego twierdzenia do gładkiej funkcji $e^x$ uzyskujemy przybliżenie szeregiem
$$
e^x = 1 + \frac{x}{1!} + \frac{x^2}{2!} + \frac{x^3}{3!} + \ldots
$$

Zatem aby obliczyć $e$ możemy szukać przybliżeń szeregiem
$$
e = 1 + \frac{1}{1!} + \frac{1}{2!} + \frac{1}{3!} + \ldots
$$

In [7]:
def e_taylor_series(n):
    sum = 0
    temp_factorial =1;
    for i in range(1,n): 
        temp_factorial *= i
        sum += 1/temp_factorial
    return sum+1
for i in range(1,21):
    print(f'Przybliżenie {i} wynosi {e_taylor_series(i)}')

Przybliżenie 1 wynosi 1
Przybliżenie 2 wynosi 2.0
Przybliżenie 3 wynosi 2.5
Przybliżenie 4 wynosi 2.666666666666667
Przybliżenie 5 wynosi 2.7083333333333335
Przybliżenie 6 wynosi 2.716666666666667
Przybliżenie 7 wynosi 2.718055555555556
Przybliżenie 8 wynosi 2.7182539682539684
Przybliżenie 9 wynosi 2.71827876984127
Przybliżenie 10 wynosi 2.7182815255731922
Przybliżenie 11 wynosi 2.718281801146385
Przybliżenie 12 wynosi 2.718281826198493
Przybliżenie 13 wynosi 2.7182818282861687
Przybliżenie 14 wynosi 2.7182818284467594
Przybliżenie 15 wynosi 2.71828182845823
Przybliżenie 16 wynosi 2.718281828458995
Przybliżenie 17 wynosi 2.7182818284590424
Przybliżenie 18 wynosi 2.7182818284590455
Przybliżenie 19 wynosi 2.7182818284590455
Przybliżenie 20 wynosi 2.7182818284590455


Obserwujemy, że już od zsumowanie 18 elementów pozwoliło nam osiągnąć stabilną dokładność przybliżenia liczby $e$. Wynik jest znacząco lepszy od metody ciągowej jednak pamiętajmy o ilości obliczeń. Tu aby policzyć n-te przybliżenia należy wykonać (n-1) mnożeń w silnii, (n-1) podzieleń oraz n sum. Zatem ilość operacji to około 3n operacji zmienno przecinkowych. Zatem ilość operacji przy 18 iteracji to około 54 tymczasem, czyli jest to bliskie iteracji 54 ciągowo

In [8]:
 print(f'Przybliżenie {18} wynosi {e_taylor_series(18)}')

Przybliżenie 18 wynosi 2.7182818284590455


In [9]:
print(f'Przybliżenie {54} wynosi {e_ciagiem(54)}')

Przybliżenie 54 wynosi 2.6935323893818506


## Ułamek łańcuchowy

Ciekawą metodą przybliżania liczby e jest ułamek łańcuchowy. Okazuje się, że bardzo wiele liczb niewymiernych posiada postać tzw. ułamka łańcuchowego (**Tak naprawdę wszystkie liczby rzeczywiste mają taką postać, kwestia jednak podania formuły do rozwijania go w taki ułamek**). Dla liczby $e$ udowodniony jest związek z następującym ułamkiem

$$
e = 2 + \frac{1}{1+ \frac{1}{2 + \frac{2}{3 + \frac{3}{4 + \frac{4}{\ldots}}} } }
$$

Przyjrzyjmy się czy z tej postaci udaje nam się uzyskać zadowalającą dokładność


In [10]:
def e_continued_fraction(n): 
    tmp = n
    for i in reversed(range(1,n)): 
        tmp = i + i / tmp
    return 2+1/tmp

for i in range(1,21):
    print(f'Przybliżenie {i} wynosi {e_continued_fraction(i)}')

Przybliżenie 1 wynosi 3.0
Przybliżenie 2 wynosi 2.6666666666666665
Przybliżenie 3 wynosi 2.7272727272727275
Przybliżenie 4 wynosi 2.7169811320754715
Przybliżenie 5 wynosi 2.7184466019417477
Przybliżenie 6 wynosi 2.718263331760264
Przybliżenie 7 wynosi 2.71828369389345
Przybliżenie 8 wynosi 2.7182816576664037
Przybliżenie 9 wynosi 2.7182818427778273
Przybliżenie 10 wynosi 2.7182818273518743
Przybliżenie 11 wynosi 2.718281828538486
Przybliżenie 12 wynosi 2.718281828453728
Przybliżenie 13 wynosi 2.7182818284593786
Przybliżenie 14 wynosi 2.7182818284590256
Przybliżenie 15 wynosi 2.7182818284590464
Przybliżenie 16 wynosi 2.718281828459045
Przybliżenie 17 wynosi 2.7182818284590455
Przybliżenie 18 wynosi 2.7182818284590455
Przybliżenie 19 wynosi 2.7182818284590455
Przybliżenie 20 wynosi 2.7182818284590455


Jak widzimy tu uzyskaliśmy dobre przylbliżenie już w 17tej iteracji. Oczywiście musimy uwzględnić złożoność obliczeń. Tu w każdej iteracji mamy 1 operację dodawania oraz dzielenia. Daje to łącznie 2n operacji. Zatem 17 iteracja przy złożoności 2n. Istotnie pokonujemy tu wzór z szeregu Taylora który potrzebował 18 operacji przy złożoności 3n.

```{python}
def e_continued_fraction(n): #2n+2
    tmp = n
    for i in reversed(range(1,n)): 
        tmp = i + i / tmp
    return 2+1/tmp
```


## Wzór Brothera

Wsród technik obliczania przybliżenia liczby e najczęściej wymienia się wzór Brothera postaci

$$
e = 2 \sum_{i=1}^{\infty} \frac{i+1}{(2i+1)!}
$$

In [11]:
def e_brother_formula(n):
    sum = 1
    temp_factorial = 1
    for i in range(1,n):
        temp_factorial *= 2*i
        temp_factorial *= (2*i+1)
        sum += (i+1) / temp_factorial
    return 2*sum

for i in range(1,21):
    print(f'Przybliżenie {i} wynosi {e_brother_formula(i)}')

Przybliżenie 1 wynosi 2
Przybliżenie 2 wynosi 2.6666666666666665
Przybliżenie 3 wynosi 2.7166666666666663
Przybliżenie 4 wynosi 2.718253968253968
Przybliżenie 5 wynosi 2.718281525573192
Przybliżenie 6 wynosi 2.7182818261984925
Przybliżenie 7 wynosi 2.7182818284467585
Przybliżenie 8 wynosi 2.718281828458994
Przybliżenie 9 wynosi 2.7182818284590446
Przybliżenie 10 wynosi 2.7182818284590446
Przybliżenie 11 wynosi 2.7182818284590446
Przybliżenie 12 wynosi 2.7182818284590446
Przybliżenie 13 wynosi 2.7182818284590446
Przybliżenie 14 wynosi 2.7182818284590446
Przybliżenie 15 wynosi 2.7182818284590446
Przybliżenie 16 wynosi 2.7182818284590446
Przybliżenie 17 wynosi 2.7182818284590446
Przybliżenie 18 wynosi 2.7182818284590446
Przybliżenie 19 wynosi 2.7182818284590446
Przybliżenie 20 wynosi 2.7182818284590446


Widzimy, że wzór ostatecznie uzyskał gorszą dokładność niż poprzednie, ale za to w 9 iteracjach. Podobną skuteczność wykazuje poprzezdni algorytm przy 15tej iteracji. Złożoność obliczeniowa wzoru Brothera wynosi 4n, 3 operacje mnożenia oraz 1 operacja sumowania

```{python}
def e_brother_formula(n):
    sum = 1
    temp_factorial = 1
    for i in range(1,n):
        temp_factorial *= 2*i
        temp_factorial *= (2*i+1)
        sum += (i+1) / temp_factorial
    return 2*sum
```

# Zmiana liczb cyfr znaczących

Zauważmy, że nasze obecne prace są mocno ograniczone przez to co dostarcza nam arytmetyka dostępna zmiennym float. Na nasze szczęście w każdym języku programowania istnieją biblioteki które pozwalają na tworzenie zmiennych o dowolnie dużych precyzjach obliczeniowych. W przypadku Python jedną z takich bibliotek jest biblioteka decimal. Poniżej przykład jak należy zmodyfikować nasze obliczenia aby uzyskać życzoną przez dokładność obliczeń (w ilości cyfr)

In [12]:
from decimal import *
getcontext().prec = 50
def e_taylor_series_decimal(n):
    sum = Decimal(0)
    temp_factorial = Decimal(1);
    for i in range(1,n):
        temp_factorial *= Decimal(i)
        sum += Decimal(1)/temp_factorial
    return sum+1
def e_continued_fraction_decimal(n):
    tmp = Decimal(n)
    for i in reversed(range(1,n)):
        tmp = Decimal(i) + Decimal(i) / tmp
    return 2+1/tmp
def e_brother_formula_decimal(n):
    sum = Decimal(1)
    temp_factorial = Decimal(1)
    for i in range(1,n):
        temp_factorial *= Decimal(2*i)
        temp_factorial *= Decimal(2*i+1)
        sum += Decimal(i+1) / temp_factorial
    return Decimal(2)*sum

In [13]:
for i in range(10,15):
    print(f'''Przybliżenie {i} \nwynosi {e_taylor_series_decimal(i)} \nwynosi {e_continued_fraction_decimal(i)}\nwynosi {e_brother_formula_decimal(i)}    ''')

Przybliżenie 10 
wynosi 2.7182815255731922398589065255731922398589065255732 
wynosi 2.7182818273518744088075986743931517096224602330743
wynosi 2.7182818284590452349287527283351994002986043726966    
Przybliżenie 11 
wynosi 2.7182818011463844797178130511463844797178130511464 
wynosi 2.7182818285384861664135778815996451660083959085657
wynosi 2.7182818284590452353593574317298071473772510089138    
Przybliżenie 12 
wynosi 2.7182818261984928651595318261984928651595318261985 
wynosi 2.7182818284537281836930799163720348764381457017809
wynosi 2.7182818284590452353602857925707585115463030677774    
Przybliżenie 13 
wynosi 2.7182818282861685639463417241195018972796750574528 
wynosi 2.7182818284593787158742820055731316238240389442054
wynosi 2.7182818284590452353602874687778324515093860784392    
Przybliżenie 14 
wynosi 2.7182818284467590023145578701134256689812245367801 
wynosi 2.7182818284590255576129561867650242693628330632633
wynosi 2.7182818284590452353602874713492656133721389999984    


*Ciekawostka - biblioteki do obliczeń ze zmienną precyzją wykorzystują tablice małych liczb zamiast dużych typów zmiennych. Tablice oczywiście mogą się zdecydowanie bardziej rozszerzać, jednak występuje tutaj znaczące marnotrawstwo pamięci komputera oraz zwolnienie w zakresie obliczeń*

In [14]:
%timeit -n100 e_taylor_series_decimal(1000)

1.14 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit -n100 e_taylor_series(1000)

440 µs ± 7.85 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Liczba $\pi$

## Metoda Archimedesa

Drugą z ważnych stałych jest liczba Pi. Tu podobnie do naszej dyspozycji odnaleźć można wiele użytecznych formuł na wyrażenie jej wartości.

Pierwszy z pomysłów pochodzi od samego Archimedesa, czyli mam ponad 2200 lat. Aby przybliżyć wartość liczby należy użyć (jak to u Archimedesa) koła. Najlepiej takiego o promieniu 1, wtedy jego pole będzie wynosić dokładnie $\pi$. W koło możemy wpisać wielokąt foremny. Im więcej boków będzie on miał, tym lepiej wypełniał on będzie koło. 

Dużo trudniej uzasadnić jak znaleźć pole takiego n-kąta foremnego, ale i na to jest sposób. Wystarczy zaznaczyć środek koła, narysować promienie do każdego wierzchołka n-kąta - by ujrzeć że jest to po prostu n trójkątów. 

![](http://www.matematykam.pl/images/l17e10.jpg)

![](http://www.matematykam.pl/images/l17e10.jpg)
Pole pojedynczego trójkąta jest wyrażone np. wzorem

$$
P = \frac{1}{2} \cdot a \cdot b \cdot \sin(\alpha_{a,b}),
$$
gdzie $a,b$ są dwoma bokami trójkąta a $\alpha_{a,b}$ jest kątem płaskim pomiędzy tymi bokami. 

Stąd nasza granica będzie mieć postać

$$
\pi = \lim_{n \to \infty} n \cdot \left( \frac{1}{2} \cdot 1 \cdot 1 \sin\left(\frac{2\pi}{n}\right) \right)
$$

In [16]:
def pi_ciagowo(n):
    return 0.5 * n * math.sin(2*math.pi / n)
for i in range(20,40):
    print(f'Przybliżenie {i} wynosi {pi_ciagowo(i)}')

Przybliżenie 20 wynosi 3.090169943749474
Przybliżenie 21 wynosi 3.094929331314494
Przybliżenie 22 wynosi 3.0990581252557265
Przybliżenie 23 wynosi 3.1026628683057793
Przybliżenie 24 wynosi 3.105828541230249
Przybliżenie 25 wynosi 3.108623589560685
Przybliżenie 26 wynosi 3.1111036357382504
Przybliżenie 27 wynosi 3.1133142550229422
Przybliżenie 28 wynosi 3.1152930753884016
Przybliżenie 29 wynosi 3.117071383059849
Przybliżenie 30 wynosi 3.11867536226639
Przybliżenie 31 wynosi 3.120127061374231
Przybliżenie 32 wynosi 3.121445152258052
Przybliżenie 33 wynosi 3.122645531946768
Przybliżenie 34 wynosi 3.1237418028816957
Przybliżenie 35 wynosi 3.1247456589761415
Przybliżenie 36 wynosi 3.125667198004746
Przybliżenie 37 wynosi 3.126515175954208
Przybliżenie 38 wynosi 3.127297215333944
Przybliżenie 39 wynosi 3.128019976726325


Podejście powyżej okazuje się być jednak bardzo wolno zbieżne. Poza tym jest obciążone jeszcze dodatkowymi absurdami. Aby policzyć wartość tej granicy trzeba bowiem 

* umieć porachować wartość funkcji sinus,
* znać wartość liczby $\pi$ (czyli to czego nam potrzeba).

$$
\pi = \lim_{n \to \infty} n \cdot \left( \frac{1}{2} \cdot 1 \cdot 1 \sin\left(\frac{2\pi}{n}\right) \right)
$$

Aby zupełnie nie zdeprecjonować tego podejścia dodajmy, że w praktyce używa się tam znanego kąta początkowego a następnie korzysta się z funkcji na połowę kąta dokładnie pisząc

$$
\sin \left( \frac{\alpha}{2} \right) = \sqrt{\frac{1-\cos \alpha}{2}},
$$

w połączeniu z 
$$
\cos \alpha = \sqrt{1 - \sin^2 \alpha}
$$

Wtedy startując np. od sześciokąta (mamy wtedy $\alpha = 60^o$) jesteśmy w stanie obliczać przybliżenia liczby $\pi$ o ile tylko umiemy przybliżać wyniki pierwiastkowania (do czego wrócimy jeszcze później). Nie mniej skupmy się dalej na szybszych metodach.

## Ułamek łańcuchowy

Również dla liczby $\pi$ można odszukać rozwinięcie w ułamek okresowy. (*Co jest bardzo ciekawe, że dla tych dwóch specjalnych stałych znamy te rozwinięcia*)

$$
\pi = 3 + \frac{1^2}{6 + \frac{3^2}{6 + \frac{5^2}{6 + \frac{7^2}{6+\ldots}}} }
$$

In [17]:
def pi_continued_fraction(n):
    tmp = 6
    for i in reversed(range(1,n)):
        tmp = 6 + (2*i+1)**2 / tmp
    return 3+1/tmp

for i in range(20,40):
    print(f'Przybliżenie {i} wynosi {pi_continued_fraction(i)}')

Przybliżenie 20 wynosi 3.1415657346585473
Przybliżenie 21 wynosi 3.141616071918187
Przybliżenie 22 wynosi 3.141572154482965
Przybliżenie 23 wynosi 3.1416106990404735
Przybliżenie 24 wynosi 3.141576685435031
Przybliżenie 25 wynosi 3.14160685134755
Przybliżenie 26 wynosi 3.141579973962182
Przybliżenie 27 wynosi 3.141604023986232
Przybliżenie 28 wynosi 3.141582418247748
Przybliżenie 29 wynosi 3.1416019000322795
Przybliżenie 30 wynosi 3.1415842726746206
Przybliżenie 31 wynosi 3.141600273698686
Przybliżenie 32 wynosi 3.1415857049341174
Przybliżenie 33 wynosi 3.1415990074057163
Przybliżenie 34 wynosi 3.141586828621159
Przybliżenie 35 wynosi 3.141598006801351
Przybliżenie 36 wynosi 3.1415877225444917
Przybliżenie 37 wynosi 3.1415972057118697
Przybliżenie 38 wynosi 3.1415884425452116
Przybliżenie 39 wynosi 3.141596556793832


## Iloczyn Viete'y

Kolejną technika wykorzystuje wzór Viete dla liczby $\pi$ postaci

$$
\frac{2}{\pi} = \frac{\sqrt{2}}{2} \cdot \frac{\sqrt{2+\sqrt{2}}}{2} \cdot \frac{\sqrt{2+\sqrt{2+\sqrt{2}}}}{2}\cdot \ldots
$$

In [18]:
def pi_viete(n):
    root = 0
    result = 1
    for i in range(n):
        root = math.sqrt(2+root) # używamy skomplikowanej funkcji
        result *= root
        result /= 2
    return 2/result
for i in range(10,30):
    print(f'Przybliżenie {i} wynosi {pi_viete(i)}')

Przybliżenie 10 wynosi 3.1415914215111997
Przybliżenie 11 wynosi 3.1415923455701176
Przybliżenie 12 wynosi 3.141592576584872
Przybliżenie 13 wynosi 3.1415926343385627
Przybliżenie 14 wynosi 3.141592648776985
Przybliżenie 15 wynosi 3.1415926523865907
Przybliżenie 16 wynosi 3.141592653288993
Przybliżenie 17 wynosi 3.1415926535145933
Przybliżenie 18 wynosi 3.1415926535709935
Przybliżenie 19 wynosi 3.1415926535850933
Przybliżenie 20 wynosi 3.1415926535886185
Przybliżenie 21 wynosi 3.1415926535895
Przybliżenie 22 wynosi 3.1415926535897203
Przybliżenie 23 wynosi 3.1415926535897754
Przybliżenie 24 wynosi 3.141592653589789
Przybliżenie 25 wynosi 3.1415926535897927
Przybliżenie 26 wynosi 3.1415926535897936
Przybliżenie 27 wynosi 3.141592653589794
Przybliżenie 28 wynosi 3.141592653589794
Przybliżenie 29 wynosi 3.141592653589794


## Wzór Wallis'a

Kolejny pomysł na obliczenia opiera się o tzw. wzór Wallisa postaci

$$
\frac{\pi}{2} = \frac{2}{1} \cdot \frac{2}{3} \cdot \frac{4}{3} \cdot \frac{4}{5} \cdot \frac{6}{5} \cdot \frac{6}{7} \cdot \ldots
$$


In [19]:
def pi_wallis(n):
    result = 1
    for i in range(1,n+1):
        result *= (2*i) * (2*i) / (2*i-1) / (2*i+1)
    return result * 2
for i in range(30,50):
    print(f'Przybliżenie {i} wynosi {pi_wallis(i)}')

Przybliżenie 30 wynosi 3.1159482858879617
Przybliżenie 31 wynosi 3.1167590973076567
Przybliżenie 32 wynosi 3.1175202106403326
Przybliżenie 33 wynosi 3.1182360591387575
Przybliżenie 34 wynosi 3.1189105640185195
Przybliżenie 35 wynosi 3.119547206305521
Przybliżenie 36 wynosi 3.120149086916423
Przybliżenie 37 wynosi 3.120718977160609
Przybliżenie 38 wynosi 3.1212593613990784
Przybliżenie 39 wynosi 3.121772473245437
Przybliżenie 40 wynosi 3.1222603264214404
Przybliżenie 41 wynosi 3.1227247411658134
Przybliżenie 42 wynosi 3.1231673669264324
Przybliżenie 43 wynosi 3.1235897019321017
Przybliżenie 44 wynosi 3.123993110133307
Przybliżenie 45 wynosi 3.1243788359155187
Przybliżenie 46 wynosi 3.124748016919408
Przybliżenie 47 wynosi 3.125101695246167
Przybliżenie 48 wynosi 3.1254408272803773
Przybliżenie 49 wynosi 3.125766292325393


## Szereg Gregory-Leibniza

Ostatnim z najistotniejszych technik przybliża liczby $\pi$ jest tzw. Szereg naprzemienny Gregory - Leibniza postaci

$$
\frac{\pi}{4} = 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} \pm \ldots
$$

In [20]:
def pi_gregory_leibniz(n):
    result = 0
    for i in range(n):
        result += 1/(4*i+1)
        result -= 1/(4*i+3)
    return result * 4
for i in range(30,50):
    print(f'Przybliżenie {i} wynosi {pi_gregory_leibniz(i)}')

Przybliżenie 30 wynosi 3.1249271439289967
Przybliżenie 31 wynosi 3.125464669965414
Przybliżenie 32 wynosi 3.125968606973288
Przybliżenie 33 wynosi 3.1264420077662343
Przybliżenie 34 wynosi 3.126887566106529
Przybliżenie 35 wynosi 3.127307667981234
Przybliżenie 36 wynosi 3.127704434335447
Przybliżenie 37 wynosi 3.1280797568782575
Przybliżenie 38 wynosi 3.1284353282369843
Przybliżenie 39 wynosi 3.128772667473754
Przybliżenie 40 wynosi 3.129093141775721
Przybliżenie 41 wynosi 3.129397984972002
Przybliżenie 42 wynosi 3.129688313406043
Przybliżenie 43 wynosi 3.129965139593801
Przybliżenie 44 wynosi 3.1302293840198954
Przybliżenie 45 wynosi 3.1304818853613083
Przybliżenie 46 wynosi 3.1307234093778527
Przybliżenie 47 wynosi 3.1309546566679236
Przybliżenie 48 wynosi 3.131176269454982
Przybliżenie 49 wynosi 3.1313888375431977


# Liczba $\sqrt{2}$

## Ułamek łańcuchowy
Omówmy też po krótce techniki odszukania wartości związanych z pierwiastkami

Mamy ułamek łańcuchowy
$$
\sqrt{2} = 1 + \frac{1}{2+\frac{1}{2+\frac{1}{2+\frac{1}{2+\ldots}}}}
$$

In [21]:
def sqrt2_continued_fraction(n):
    tmp = 2
    for i in reversed(range(1,n)):
        tmp = 2 + 1 / tmp
    return 1+1/tmp

for i in range(5,25):
    print(f'Przybliżenie {i} wynosi {sqrt2_continued_fraction(i)}')

Przybliżenie 5 wynosi 1.4142857142857144
Przybliżenie 6 wynosi 1.4142011834319526
Przybliżenie 7 wynosi 1.4142156862745099
Przybliżenie 8 wynosi 1.4142131979695431
Przybliżenie 9 wynosi 1.4142136248948696
Przybliżenie 10 wynosi 1.4142135516460548
Przybliżenie 11 wynosi 1.4142135642135643
Przybliżenie 12 wynosi 1.4142135620573204
Przybliżenie 13 wynosi 1.4142135624272734
Przybliżenie 14 wynosi 1.4142135623637995
Przybliżenie 15 wynosi 1.4142135623746899
Przybliżenie 16 wynosi 1.4142135623728214
Przybliżenie 17 wynosi 1.414213562373142
Przybliżenie 18 wynosi 1.414213562373087
Przybliżenie 19 wynosi 1.4142135623730965
Przybliżenie 20 wynosi 1.414213562373095
Przybliżenie 21 wynosi 1.4142135623730951
Przybliżenie 22 wynosi 1.4142135623730951
Przybliżenie 23 wynosi 1.4142135623730951
Przybliżenie 24 wynosi 1.4142135623730951


## Wzór Taylora
Dostępna jest również formuła oparta o wzór Taylora postaci:

$$
\sqrt{2} = \sum_{n=0}^\infty \frac{(2k+1)!}{(k!)^2 \cdot 2^{3k+1}}
$$

In [22]:
def sqrt2_taylor_euler(n):
    result = 0.5
    k2p1 = 1
    two_power = 2
    kp2 = 1
    for i in range(1,n):
        k2p1 *= (2*i) * (2*i+1)
        kp2 *= i*i
        two_power *= 8
        result += k2p1/kp2/two_power
    return result
for i in range(1,20):
    print(f'Przybliżenie {i} wynosi {sqrt2_taylor_euler(i)}')

Przybliżenie 1 wynosi 0.5
Przybliżenie 2 wynosi 0.875
Przybliżenie 3 wynosi 1.109375
Przybliżenie 4 wynosi 1.24609375
Przybliżenie 5 wynosi 1.322998046875
Przybliżenie 6 wynosi 1.36529541015625
Przybliżenie 7 wynosi 1.3882064819335938
Przybliżenie 8 wynosi 1.4004802703857422
Przybliżenie 9 wynosi 1.407000720500946
Przybliżenie 10 wynosi 1.4104420691728592
Przybliżenie 11 wynosi 1.4122487772256136
Przybliżenie 12 wynosi 1.4131931927986443
Przybliżenie 13 wynosi 1.4136850759095978
Przybliżenie 14 wynosi 1.4139404767556698
Przybliżenie 15 wynosi 1.4140727379081
Przybliżenie 16 wynosi 1.4141410728368555
Przybliżenie 17 wynosi 1.4141763080344951
Przybliżenie 18 wynosi 1.414194443797986
Przybliżenie 19 wynosi 1.4142037635653355


## Iteracja prosta

Najczęstszym wyborem do przybliżania pierwiastka $\sqrt{2}$ jest wykorzystanie iteracji prostej do odszukania. Zauważmy, że równanie $ x^2 = 2 $ ma rozwiązanie dodatnie właśnie $\sqrt{2}$. Jednak równanie to wymaga przekształceń aby uzyskać jego zbieżność
$$
\frac{x^2}{2} = 1 
$$

$$
x^2 = \frac{x^2}{2} + 1
$$

$$
x = \frac{x}{2} + \frac{1}{x}
$$

In [23]:
def sqrt2_iteration(n):
    a = 1.41
    for i in range(n):
          a = a/2 + 1/a
    return a

for i in range(1,20):
    print(f'Przybliżenie {i} wynosi {sqrt2_iteration(i)}')

Przybliżenie 1 wynosi 1.4142198581560284
Przybliżenie 2 wynosi 1.4142135623871086
Przybliżenie 3 wynosi 1.414213562373095
Przybliżenie 4 wynosi 1.414213562373095
Przybliżenie 5 wynosi 1.414213562373095
Przybliżenie 6 wynosi 1.414213562373095
Przybliżenie 7 wynosi 1.414213562373095
Przybliżenie 8 wynosi 1.414213562373095
Przybliżenie 9 wynosi 1.414213562373095
Przybliżenie 10 wynosi 1.414213562373095
Przybliżenie 11 wynosi 1.414213562373095
Przybliżenie 12 wynosi 1.414213562373095
Przybliżenie 13 wynosi 1.414213562373095
Przybliżenie 14 wynosi 1.414213562373095
Przybliżenie 15 wynosi 1.414213562373095
Przybliżenie 16 wynosi 1.414213562373095
Przybliżenie 17 wynosi 1.414213562373095
Przybliżenie 18 wynosi 1.414213562373095
Przybliżenie 19 wynosi 1.414213562373095


# Inne funkcje

# Obliczenia trudnych funkcji

W poprzedniej części obserwowaliśmy różne sposoby na przybliżanie liczb. Dostępna nam tam była mnogość metod. Wyobraźmy sobie jak dużo bardziej różnorodnym może być zadanie przybliżania funkcji. Tymczasem okazuje się, że w niemal każda technika opiera się na stosowaniu wzoru Taylora:

**Twierdzenie Taylora**
*Niech $[a,b] \subset \mathbb{R} $ pewien przedział oraz $ f: [a,b] \to \mathbb{R}$ będzie funkcją $(n+1)$ różniczkowalną na przedziale $[a,b]$ w sposób ciągły. Wówczas dla każdego punktu z przedziału $(a,b)$ spełniony jest następujący wzór
$$
f(x) = f(a) + \frac{x-a}{1!} f^{(1)}(a) + \frac{(x-a)^2}{2!} f^{(2)}(a) + \ldots + \frac{(x-a)^n}{n!} + \frac{(x-a)^{n+1}}{(n+1)!} f^{(n+1)} (\xi),
$$
gdzie $\xi \in [x,a]$ (gdy $x < a$ lub do $[a,x]$ jeśli $x >a$)*

Przypominjmy w tym miejscu kilka z najważniejszych rozwinięć funkcji

| Funkcja &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | Rozwinięcie &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; |
| :------------- | -------------: |
| $e^x$  | $ 1+ x + \frac{x^2}{2!} + \frac{x^3}{3!} + \frac{x^4}{4!} + \ldots  $  |
| $\sin(x)$  | $ x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + \ldots $  |
| $\cos(x)$  | $ 1 - \frac{x^2}{2!} + \frac{x^4}{4!} - \frac{x^6}{6!} + \ldots $  |
| $\frac{1}{1+x}$, $-1 < x < 1 $  | $ 1 - x + x^2 - x^3 + x^4 -x^5 + \ldots $  |



Oprócz tego dostępne są nam techniki oparte ułamki łańcuchowe w kilku wybranych przypadkach, np.
$$
e^x = 1 + \frac{x}{
1 - \frac{x}{
    2 + \frac{x}{
        3- \frac{x}{
            2+\frac{x}{
                5 - \frac{x}{2+\ldots}
                }
            } 
        }
    } 
}
$$
$$
\tan x = \frac{1}{
\frac{1}{x} - \frac{1}{
    \frac{3}{x} - \frac{1}{
        \frac{5}{x}- \frac{1}{
            \frac{7}{x} - \frac{1}{
                \frac{9}{x} - \ldots
                }
            } 
        }
    } 
}
$$

Przyjrzyjmy się jedynie przykładowej implementacji wybranej funkcji

In [24]:
def my_sin(x, n):
    result = x
    temp_x = x
    x_sqrt = x*x
    factor = 1
    for i in range(n):
        temp_x *= x_sqrt
        factor *= (4*i+3)*(4*i+2)
        result -= temp_x / factor
        temp_x *= x_sqrt
        factor *= (4*i+5)*(4*i+4)
        result += temp_x / factor
    return result

In [25]:
print(f'Przybliżenie sin(1) wynosi {math.sin(1)}')
for i in range(1,10):
    print(f'Przybliżenie sin(1) no {i} wynosi {my_sin(1,i)}')

Przybliżenie sin(1) wynosi 0.8414709848078965
Przybliżenie sin(1) no 1 wynosi 0.8416666666666667
Przybliżenie sin(1) no 2 wynosi 0.8414710097001764
Przybliżenie sin(1) no 3 wynosi 0.8414709848086585
Przybliżenie sin(1) no 4 wynosi 0.8414709848078965
Przybliżenie sin(1) no 5 wynosi 0.8414709848078965
Przybliżenie sin(1) no 6 wynosi 0.8414709848078965
Przybliżenie sin(1) no 7 wynosi 0.8414709848078965
Przybliżenie sin(1) no 8 wynosi 0.8414709848078965
Przybliżenie sin(1) no 9 wynosi 0.8414709848078965


# Dziękuje za uwagę